This scripts generate a GeoJSON file with the geometries of every countries.

It takes a base GeoJSON file downloaded from https://geojson-maps.kyd.au/ and merge all entries corresponding to the same country.


In [5]:
import pandas as pd
import json

# data from https://geojson-maps.kyd.au/
file = "country_10m.json"
output_file = "processed_countries_10m.json"

df = pd.read_json(file)

alt_df = pd.json_normalize(df["features"], max_level=0)

geometry = pd.json_normalize(alt_df["geometry"], max_level=0).rename(
    columns={"type": "geometry_type"}
)
prop = pd.json_normalize(alt_df["properties"])
# prop.columns
# prop[["iso_a2"]].drop_duplicates()
# df

In [6]:
data_df = pd.concat(
    [
        alt_df,
        prop[["iso_a2"]],
        geometry[["geometry_type", "coordinates"]],
    ],
    axis=1,
)


def merge_geometries(group) -> tuple[str, list]:
    concat_coord = []
    if len(group) == 1:
        geom_type = group.iloc[0]["geometry_type"]
        coordinates = group.iloc[0]["coordinates"]
        return {"geometry_type": geom_type, "coordinates": coordinates}

    for _, row in group.iterrows():
        geom_type = row["geometry_type"]
        coordinates = row["coordinates"]
        if geom_type == "MultiPolygon":
            concat_coord += coordinates
        elif geom_type == "Polygon":
            concat_coord.append(coordinates)
        else:
            raise Exception(f"Unhandled geometry type: {geom_type}")

    return {"geometry_type": "MultiPolygon", "coordinates": concat_coord}


res = []
for name, group in data_df.groupby("iso_a2"):
    temp = merge_geometries(group)
    temp["iso_a2"] = name
    res.append(temp)
res = pd.DataFrame.from_records(res)
res["properties"] = res["iso_a2"].apply(lambda x: {"iso_a2": x})
res["geometry"] = res.apply(
    lambda x: {"type": x["geometry_type"], "coordinates": x["coordinates"]},
    axis=1,
)
res["feature"] = res.apply(
    lambda x: {
        "type": "Feature",
        "properties": x["properties"],
        "geometry": x["geometry"],
    },
    axis=1,
)

new_geojson = {
    "type": "FeatureCollection",
    "features": res["feature"].to_list(),
}
new_geojson
with open(output_file, "w") as f:
    json.dump(new_geojson, f, indent=2)